In [2]:
import re
import time
import json
import os
import pandas as pd
import arrow
today_dt = arrow.now().format('YYYYMMDD')



In [3]:
import pandas as pd

In [4]:
from platform import python_version

print(python_version())

3.6.9


In [22]:
today_dt = arrow.now().format('YYYYMMDD')


In [6]:
tweets = pd.read_parquet("tweets_all." + today_dt + ".parquet")

In [7]:
tweets.head()

,created_at,extended_tweet.full_text,in_reply_to_screen_name,is_quote_status,lang,place.country,place.country_code,place.full_name,place.name,place.place_type,...,source,text,user.created_at,user.description,user.followers_count,user.friends_count,user.location,user.name,user.screen_name,user.verified
0,Sun Mar 29 14:49:56 +0000 2020,None,None,0.0,es,None,None,None,None,None,...,"<a href=""http://twitter.com/download/iphone"" r...",RT @teresitaradiol: Los mexicanos no creen en ...,Wed Feb 27 21:39:34 +0000 2013,Crimmigration strategist. Power to the people....,1111.0,714.0,None,SV,arreenitta,0.0
1,Sun Mar 29 14:49:56 +0000 2020,None,None,1.0,en,None,None,None,None,None,...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @iamkarendavila: TIME IS OF THE ESSENCE. Dr...,Wed Dec 19 23:54:55 +0000 2018,Christian - Lutheran\n\n\n\n\nEach day is a gi...,6.0,157.0,None,...,eysibi12,0.0
2,Sun Mar 29 14:49:56 +0000 2020,None,None,0.0,es,None,None,None,None,None,...,"<a href=""http://twitter.com/download/android"" ...",RT @miguelhotero: Curioso que en Cuba y Venezu...,Sat Oct 19 01:47:22 +0000 2013,"Ing quimico ULA,amante de mi esposa padre de m...",27.0,152.0,None,Argentino Y,afryka53,0.0
3,Sun Mar 29 14:49:56 +0000 2020,None,None,0.0,in,None,None,None,None,None,...,"<a href=""http://twitter.com/download/iphone"" r...","RT @aainaaqila: Scammer: Saya call dari SPRM, ...",Sat Nov 21 07:07:41 +0000 2015,:(,1832.0,490.0,"Kota Kinabalu, Perlis",kerol #DudukRumahMak,khairidzwan,0.0
4,Sun Mar 29 14:49:56 +0000 2020,None,None,0.0,es,None,None,None,None,None,...,"<a href=""http://twitter.com/download/iphone"" r...","RT @cesarlitardo: El covid, le gano la batalla...",Mon Jun 07 17:29:12 +0000 2010,Abogada Laboralista de UniNorte,428.0,942.0,"ÜT: 9.2968493,-75.3906836",Tatiana Barrios A,TatianaBAlvarez,0.0


In [8]:
def strip_links(text):
    link_regex    = re.compile('((http?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def strip_all_entities(text):
    import string
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            if separator == "'":
                text = text.replace(separator,'')
            else:
                text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if (word[0] not in entity_prefixes) and (word != 'RT'):
                words.append(word)    
    return ' '.join(words)

def strip_numbers(text):
    text = re.sub('[0-9]+', '', text)
    return text

def strip_junk(text):
    text = re.sub(r'\W+', ' ', text)
    return text

from nltk.stem import PorterStemmer
ps = PorterStemmer()
# 7. Use stemming technique to return words backs to its root form (avoid using aggressive
# stemming)
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [ps.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)


def preprocess(text_data):
    preprocessed_text=[]
    for text in text_data:
        #regext to remove RT retweets
        text=re.sub('RT'," ",text)
        #regex to remove punctuations,hashtags,@usermentions,URLs and convert to lowercase
        text=re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).lower()
        #regex to remove numbers
        text=re.sub("\d+", " ", text)
        preprocessed_text.append(text)
    return preprocessed_text


In [9]:
tweets['text_uni'] = tweets['text'].astype('unicode')


In [10]:
#tweets['text_str']= tweets['text_uni'].astype(str)
tweets['text_str'] = tweets['text_uni'].str.encode('ascii', 'ignore').str.decode('ascii')

In [11]:
def make_lower(text):
    text = text.lower()
    return text

tweets['text_clean'] = tweets['text_str'].apply(strip_links).apply(strip_all_entities) \
        .apply(strip_numbers).apply(make_lower) \
        .apply(strip_junk).apply(stem_sentences)


In [12]:
tweets

,created_at,extended_tweet.full_text,in_reply_to_screen_name,is_quote_status,lang,place.country,place.country_code,place.full_name,place.name,place.place_type,...,user.description,user.followers_count,user.friends_count,user.location,user.name,user.screen_name,user.verified,text_uni,text_str,text_clean
0,Sun Mar 29 14:49:56 +0000 2020,None,None,0.0,es,None,None,None,None,None,...,Crimmigration strategist. Power to the people....,1111.0,714.0,None,SV,arreenitta,0.0,RT @teresitaradiol: Los mexicanos no creen en ...,RT @teresitaradiol: Los mexicanos no creen en ...,lo mexicano no creen en el coronaviru aaaah pe...
1,Sun Mar 29 14:49:56 +0000 2020,None,None,1.0,en,None,None,None,None,None,...,Christian - Lutheran\n\n\n\n\nEach day is a gi...,6.0,157.0,None,...,eysibi12,0.0,RT @iamkarendavila: TIME IS OF THE ESSENCE. Dr...,RT @iamkarendavila: TIME IS OF THE ESSENCE. Dr...,time is of the essenc dr rustico jimenez call ...
2,Sun Mar 29 14:49:56 +0000 2020,None,None,0.0,es,None,None,None,None,None,...,"Ing quimico ULA,amante de mi esposa padre de m...",27.0,152.0,None,Argentino Y,afryka53,0.0,RT @miguelhotero: Curioso que en Cuba y Venezu...,RT @miguelhotero: Curioso que en Cuba y Venezu...,curioso que en cuba y venezuela son igual la c...
3,Sun Mar 29 14:49:56 +0000 2020,None,None,0.0,in,None,None,None,None,None,...,:(,1832.0,490.0,"Kota Kinabalu, Perlis",kerol #DudukRumahMak,khairidzwan,0.0,"RT @aainaaqila: Scammer: Saya call dari SPRM, ...","RT @aainaaqila: Scammer: Saya call dari SPRM, ...",scammer saya call dari sprm ini bercakap denga...
4,Sun Mar 29 14:49:56 +0000 2020,None,None,0.0,es,None,None,None,None,None,...,Abogada Laboralista de UniNorte,428.0,942.0,"ÜT: 9.2968493,-75.3906836",Tatiana Barrios A,TatianaBAlvarez,0.0,"RT @cesarlitardo: El covid, le gano la batalla...","RT @cesarlitardo: El covid, le gano la batalla...",el covid le gano la batalla a mi madr seguro d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64250,Sun Mar 29 15:33:02 +0000 2020,None,None,0.0,in,None,None,None,None,None,...,-H-CASB-IPROMNIAN-JOHANNIAN-MANUTD,421.0,214.0,None,FDA,faiqfikrey,0.0,RT @KKMPutrajaya: TERKINI: Dukacita dimaklumka...,RT @KKMPutrajaya: TERKINI: Dukacita dimaklumka...,terkini dukacita dimaklumkan terdapat satu lag...
64251,Sun Mar 29 15:33:02 +0000 2020,None,None,0.0,en,None,None,None,None,None,...,Just barely learning to balance // Fer vel í v...,991.0,1548.0,Afghanistan,Brynja Oskarsdottir,BrynjaHuld,0.0,RT @nicktolhurst: 4/\n\nWorth a read also:\n\n...,RT @nicktolhurst: 4/\n\nWorth a read also:\n\n...,worth a read also http t co ypudjphzz
64252,Sun Mar 29 15:33:02 +0000 2020,None,None,0.0,en,None,None,None,None,None,...,"Use our voice for kindness, our ears for compa...",1088.0,584.0,Kuala Lumpur ~ Bangkok - Dili,Hazel Sali,lady_chantek,0.0,"RT @aafaizli: America @realDonaldTrump, we hum...","RT @aafaizli: America @realDonaldTrump, we hum...",america we humbl urg you to follow china south...
64253,Sun Mar 29 15:33:02 +0000 2020,None,None,0.0,it,None,None,None,None,None,...,sono talmente prolisso che uso Twitter per imp...,251.0,537.0,Milano - Italy,Claudio,LenzReuven,0.0,RT @disinformatico: @matteosalvinimi Fonti: Uf...,RT @disinformatico: @matteosalvinimi Fonti: Uf...,fonti ufficio sviluppo economico del canton ti...


In [13]:
tweets.columns = tweets.columns.str.lower()
tweets['word_list'] = tweets['text_clean'].str.split()


In [14]:
# Vocabulary time
vocab_dict = {}
for word_list in tweets['word_list']:
    for word in word_list:
        vocab_dict[word] = vocab_dict.get(word, 0) + 1
# K now clean up that list by eliminating the stopwords


In [15]:
# 8. Remove stopwords by using from nltk.corpus import stopwords,
# stopwords are dumb words like "the" and "a" 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
stopwords=set(stopwords.words('english'))
vocab_clean_dict = {k:v for k,v in vocab_dict.items() if v>1 and k not in stopwords}


In [17]:
# The bag of words is too dam big and crashes the computer
# So let's only allow words with 10 or more instances 
vocab_toosmall_dict = {k:v for k,v in vocab_clean_dict.items() if v<10}
vocab_trimmed_dict = {k:v for k,v in vocab_clean_dict.items() if v>=10}


In [18]:
pat = r'\b(?:{})\b'.format('|'.join(vocab_toosmall_dict.keys()))
tweets['text_trim'] = tweets['text_clean'].str.replace(pat, '')


In [19]:
pd.options.display.max_colwidth = 240

tweets[['text','text_trim']]




,text,text_trim
0,RT @teresitaradiol: Los mexicanos no creen en el Coronavirus... \n\nAaaah pero díganles que apareció una virgen en una tortilla.,lo mexicano no creen en el coronaviru aaaah pero dganl que apareci una virgen en una tortilla
1,RT @iamkarendavila: TIME IS OF THE ESSENCE. Dr Rustico Jimenez calls on DOH to skip the bureaucracy of having to write a letter to request…,time is of the dr rustico call on doh to the of have to write a letter to request
2,RT @miguelhotero: Curioso que en Cuba y Venezuela son iguales las cifras oficiales de infectados por coronavirus https://t.co/qALSCr8Brx,curioso que en cuba y venezuela son igual la cifra oficial de infectado por coronaviru http t co
3,"RT @aainaaqila: Scammer: Saya call dari SPRM, ini bercakap dengan Puan Nur Aainaa..(proceeds with typical scammer talk)\n\nMe: ...kita sekara…",scammer saya call dari sprm ini bercakap dengan puan nur aainaa proce with typic scammer talk me kita sekara
4,"RT @cesarlitardo: El covid, le gano la batalla a mi madre, seguro Dios tiene un lugar a su lado para acogerla en su seno 🙌",el covid le gano la batalla a mi madr seguro dio tien un lugar a su lado para acogerla en su seno
...,...,...
64250,RT @KKMPutrajaya: TERKINI: Dukacita dimaklumkan terdapat satu lagi kematian di negara kita akibat #COVID19 menjadikan jumlah keseluruhan s…,terkini dukacita dimaklumkan terdapat satu lagi kematian di negara kita akibat menjadikan jumlah keseluruhan s
64251,RT @nicktolhurst: 4/\n\nWorth a read also:\n\nhttps://t.co/YpUd0JPHzz,worth a read also http t co ypudjphzz
64252,"RT @aafaizli: America @realDonaldTrump, we humbly urge you to follow China, South Korea and Malaysia on how to deal with #coronavirus #covi…",america we humbl urg you to follow china south korea and malaysia on how to deal with
64253,RT @disinformatico: @matteosalvinimi Fonti: Ufficio Sviluppo Economico del Canton Ticino: https://t.co/TtWOq0x6lj Procedura online: https:/…,fonti ufficio sviluppo del http t co ttwoqxlj procedura onlin http


In [20]:
#  10 - Now, use the scikit-learn function CountVectorizer to create bag-of-words features        
import scipy.sparse
from sklearn.feature_extraction.text import CountVectorizer


In [21]:
vectorizer=CountVectorizer()
vectorizer.fit(vocab_trimmed_dict)

tweets['bag_of_words'] = list(vectorizer.transform(tweets['text_trim']).todense())




In [26]:
import nltk
from nltk.corpus import stopwords

stop = stopwords.words('english')


def tokenizeit(text_data):
    tokenized_text=[]
    for text in text_data:
        #tokenize the text
        tokenized=nltk.word_tokenize(text)
        #remove stop words
        tokenized_text.append(" ".join(list(x for x in tokenized if x not in stop)))
    return tokenized_text


tweets['text_tokenized']=tokenizeit(tweets['text_trim'])


In [57]:
for num_words in [100,200,400,800]: 
    vectorizer = CountVectorizer(max_features = num_words, stop_words='english')
    bag_of_words = vectorizer.fit_transform(tweets['text_tokenized']).toarray()

    sum_of_words=bag_of_words.sum(axis=0)

    df=pd.DataFrame(bag_of_words,columns=vectorizer.get_feature_names())
    
    df.columns = ['bow_' + str(col) for col in df.columns]

    #column names need to be str for parquet (apparently!)
    df.columns = df.columns.map(str)

#    df = tweets.append(df, ignore_index=True)
    
    df = pd.concat([tweets, df],axis=1)
    
    pretty_front = df[['text','text_tokenized']]
    pretty_back = df.drop(['text','text_tokenized'], axis=1)
    pretty_full = pd.concat([pretty_front,pretty_back], axis=1)

    #write as parquet
    pretty_full.drop(columns=['bag_of_words']).to_parquet("tweets_bow_t" + str(num_words) + "." + str(today_dt) + ".parquet")

    print(pretty_full.shape)

(64255, 138)
(64255, 238)
(64255, 438)
(64255, 838)


0        [[[[[1 0 0 ... 0 0 0]]]]]
1        [[[[[0 0 0 ... 0 0 0]]]]]
2        [[[[[0 0 0 ... 0 0 0]]]]]
3        [[[[[0 1 0 ... 0 0 0]]]]]
4        [[[[[0 0 0 ... 0 0 0]]]]]
                   ...            
64250    [[[[[0 0 0 ... 0 0 0]]]]]
64251    [[[[[0 0 0 ... 0 0 0]]]]]
64252    [[[[[0 0 0 ... 0 0 0]]]]]
64253    [[[[[0 0 0 ... 0 0 0]]]]]
64254    [[[[[0 0 0 ... 0 0 0]]]]]
Name: bag_of_words, Length: 64255, dtype: object